In [ ]:
from hmmlearn import hmm

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('step2/seattle-tacoma_all_parts', header=None)

In [ ]:
df.head()

In [ ]:
df = df.sort_values([0,1], ascending=True)

In [ ]:
df.drop([1], axis=1, inplace=True)

In [ ]:
#mapping to 12 categories defined in the file mapping.csv
mapping = pd.read_csv('step2/mapping.csv', header=None, names =['Original','Mapped','None1',"None2"])

In [ ]:
mapping = mapping.drop(['None1', 'None2'], axis='columns') #.value_counts()

In [ ]:
map_dict = dict(zip(mapping['Original'],mapping['Mapped']))

In [ ]:
df2 = df.iloc[:,1:].replace(map_dict)

In [ ]:
df2 = df2.astype('category')

In [ ]:
df2 = df2.apply(lambda x: x.cat.codes)

In [ ]:
df3 = pd.concat((df.iloc[:,:1],df2),axis=1)

In [ ]:
df.columns


In [ ]:
#for adding rotations
X=df3.groupby([0]).apply(lambda x: x.iloc[:,2:].values.flatten())

In [ ]:
indexer = np.arange(95)[None, :] + 1*np.arange(15675-94)[:, None]

In [ ]:
X = X.apply(lambda x: pd.DataFrame(x[indexer]))

In [ ]:
X_list = []
for i in range(X.shape[0]):
    if(i % 500 == 0):
        print(i)
    X_list.append(X.values[i])
Big_Data = pd.concat(X_list)

In [ ]:
df2 = Big_Data

In [ ]:
#creating testing and training datasets (we want to use past interactions to predict the future interactions)
import numpy as np
first_80 = int(0.8*df2.shape[0])
train = df2.iloc[:first_80,:]
test = df2.iloc[first_80:,:]

In [ ]:
#shuffling sequences since the order doesn't matter now
train_200 = train.sample(frac=1)
train_200 = train_200.head(800000)

In [ ]:
test_200 = test.sample(frac=1)
test_200 = test_200.head(160000)

In [ ]:
#creating model using hmm package (tried various parameters)
model = hmm.MultinomialHMM(n_components=16, n_iter=10,algorithm = 'map', verbose=True)  # Viterbi is set by default as the 'algorithm' optional parameter.

In [ ]:
model.fit(train_200.T) #.to_numpy())

In [ ]:
np.set_printoptions(suppress=True)
model.emissionprob_

In [ ]:
#building transition matrix: probabilities moving to real 12 categories from 16 hodden states
import seaborn as sn
sn.set(rc={'figure.figsize':(11.7,8.27)})
np.set_printoptions(suppress=True)
sn.heatmap(model.emissionprob_,annot=True,cbar=False )

In [ ]:
#the posterior probability for the element before the last one
predict_states = model.predict_proba(np.array(test_200.iloc[:,-2]).reshape(-1,1))#drop(test_200.columns[-1],axis=1)

In [ ]:
predict_states.shape

In [ ]:
model.emissionprob_.shape

In [ ]:
#predict next item (the last one) for the element before the last one by multiplying the posterior probability matrix (160000x16) by matrix of probabilities to reach every real state from 16 hidden states (16x12)
next_items = np.dot(predict_states,np.array(model.emissionprob_))

In [ ]:
#checking the output dimenstions
next_items.shape

In [ ]:
np.set_printoptions(suppress=True)
next_items

In [ ]:
next_items.shape

In [ ]:
#choosing the element with the highest probability
next_items_argmax = np.zeros((test_200.shape[0], ))
j = 0
for row in (next_items): 
    next_items_argmax[j] = row.argmax()
    #print(row,j)
    #print(next_items_argmax[j])
    j=j+1
    if(j==test_200.shape[0]):
        break

In [ ]:
#counting number of predicted states that are not 0 (since 0 has the highest probability to go into for the whole set)
np.count_nonzero(next_items_argmax)

In [ ]:
#the actual last item in the test set
actual_items = np.array(test_200.iloc[:,-1])

In [ ]:
actual_items.shape

In [ ]:
next_items.shape

In [ ]:
#calculating the precision (summing up the number of true predictions and dividing by the number of elements)
precision = np.sum(next_items_argmax.astype(int) == actual_items)/len(next_items)

In [ ]:
precision